<a href="https://colab.research.google.com/github/dpalacioj/mlops-essentials/blob/udemy-notes/3_batching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
from tqdm.auto import tqdm

In [3]:
model = AutoModelForCausalLM.from_pretrained("TheFuzzyScientist/diabloGPT_open-instruct").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [4]:
dataset = load_dataset("hakurei/open-instruct-v1", split="train")

README.md:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

instruct_data.json:   0%|          | 0.00/104M [00:00<?, ?B/s]

additional_data.json:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

alpaca_data.json:   0%|          | 0.00/22.7M [00:00<?, ?B/s]

gpt4_data.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

roleplay_instruct.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

self_instruct.json:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

sharegpt_data.json:   0%|          | 0.00/109M [00:00<?, ?B/s]

synthetic_instruct.json:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [5]:
dataset = dataset.to_pandas()
dataset.head(2)

,output,input,instruction
0,1. Eat a balanced diet and make sure to includ...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?


In [12]:
def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(inputs, max_length=64)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated[: generated.find(".") + 1]

In [14]:
generate_text("What is the best criteria to choose a hobbie?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the best criteria to choose a hobbie?  The best criteria to choose a hobbie is to be able to find a place that is convenient for you and your family.'

Flash Attention